The Turán number of $C_5^-$
==============================

This notebook contains calculations for the Turán number of $C_5^-$

As the blowup of $K_4^-$ contains $C_5^-$, we can additionally assume 
that we work in $K_4^-$-free structures. 

1. The first cell sets up the combinatorial theory of $C_5^-$ and 
$K_4^-$-free 3-graphs (called TGp). In addition, it sets up the 
combinatorial theory on the same 3-graphs with vertices colored 
from 3 possible colors (called CTGp)

2. The second cell performs the basic calculation of upper bounding
edges in the theory. It gives the 1/4 + 1/1000 (in fact 0.250728)
upper bound. The certificate proving the claim is in the file "noc5m_0.json".

3. That upper bound is used in the following cell. It lower bounds the
quotient k222/k111 near the optimum (where this value is 24/121~0.198),
by around 0.194. The certificate proving the claim is in the file "noc5m_1a.json".

4. The next cell minimizes k222 (which is easier to argue with in the
paper) near the optimum. At the optimum, this value is 6/121~0.0495 and
the code provides the bound 0.0487 . The certificate proving the claim is in
the file "noc5m_1b.json".

5. The last cell works in the colored theory, and shows that the bad
edges have number smaller than the missing good edges. It uses the upper
bound about k222/k111. The calculations provide the precise density bound
that there are less bad edges than missing edges asymptotically.
(Even stability holds). The certificate proving this claim is in the file "noc5m_2.json".

In [ ]:
### This cell is just to set up the theory.
### In practice it is not needed, as the calculations (multiplication table and generated structures)
### are already done and saved. But it is here for completeness (and for re-runs from scratch)

from sage.algebras.flag_algebras import *

# These are helper functions, to deal with classical exclusion (not just induced)
def check_containment(smalls, larges):
    sis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in smalls]
    lis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in larges]
    res = []
    for ll in lis:
        good = True
        for ss in sis:
            for _ in ll.isomorphic_substructures_iterator(ss):
                good = False
                break
            if not good:
                break
        res.append(good)
    return res

def check_containment_cert(smalls, large):
    sis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in smalls]
    lis = IncidenceStructure(large.size(), large.blocks()['edges'])
    for ii, ss in enumerate(sis):
        for xx in lis.isomorphic_substructures_iterator(ss):
            return [xx, smalls[ii]]
    return None

# This is some hack to create the theory for 3-graphs without C5- and K4-
# up to size 7. It is easier to make them as extensions of 6 sized structures
# so this code does that.

# Reset three graphs, so nothing is excluded
TG = ThreeGraphTheory
TG.exclude()

# C5 minus
C5m = TG(5, edges=[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 0]])

# flags of size 5
fl5 = TG.generate(5)

# boolean vector indicating each element in fl5 if it has C5m
gs = check_containment([C5m], fl5)

# k4 and k4m (the two induced structures with size 4 excluded)
k4 = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]])
k4m = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]])

# set the excluded structures. k4, k4m and all in fl5 containing C5m
exls = [k4, k4m] + [xx for ii, xx in enumerate(fl5) if not gs[ii]]
TG.exclude(exls)

# check the list of flags with size 5 and 6
fl5 = TG.generate(5)
fl6 = TG.generate(6)

# quick 3-graph identifier code. This will be the identifier for
# the theory of C5- free 3-graphs (any identifier working for 3-graphs can work here)
def _identify_hypergraph(n, ftype_points, edges):
    g = Graph([list(range(n+len(edges))), [(i+n,x) for i,b in enumerate(edges) for x in b]], 
              format='vertices_and_edges')
    ftype_union = [jj for ff in ftype_points for jj in ff]
    partt = list(ftype_points) + \
            [[ii for ii in range(n) if ii not in ftype_union]] + \
            [list(range(n,n+len(edges)))]
    blocks = tuple(g.canonical_label(partition=partt).edges(labels=None, sort=True))
    return (n, tuple([len(xx) for xx in ftype_points]), blocks)

# generator code. It should really just return TG, but for size 7 that takes too long
# so this hack just returns TG for size up to 6, and for 7 it generates all flags
# with this extension technique
def _gen(n):
    if n<=4:
        for xx in TG.generate_flags(n):
            yield xx.blocks()
    elif n==5:
        for xx in fl5:
            yield xx.blocks()
    elif n==6:
        for xx in fl6:
            yield xx.blocks()
    elif n==7:
        import itertools
        from tqdm import tqdm
        fl7_m = [[] for ii in range(35+1)]
        subs = list(itertools.combinations(range(6), int(2)))
        for xx in tqdm(fl6):
            xb = xx.blocks()['edges']
            for ii in range(15+1):
                for pps in itertools.combinations(subs, int(ii)):
                    xbp = [[pp[0], pp[1], 6] for pp in pps] + xb
                    flxp = TG(7, edges=xbp)
                    en = len(xbp)
                    if flxp not in fl7_m[en]:
                        if check_containment(exls, [flxp])[0]:
                            fl7_m[en].append(flxp)
        fl7 = [yy for xx in fl7_m for yy in xx]
        for xx in fl7:
            yield xx.blocks()
    else:
        #for n>=8 just return an empty list, this will not be called so doesn't 
        #really matter
        return []

# Create the theory based on this generator and identifier
TGp = CombinatorialTheory("NoC5m", _gen, _identify_hypergraph, edges=3)

# for sanity check, print the number of structures with size 5, 6, 7
# should be 9 55 1127
print(len(TGp.generate(5)), len(TGp.generate(6)), len(TGp.generate(7)))




# This is code to create colored theories
# This is a default code for all color partition
def _identifyCT(k, order_partition, n, ftype_points, **kwargs):
    is_graph = (k==2)
    color_number = sum(len(xx) for xx in order_partition)
    edges = kwargs["edges"]
    ftype_union = [jj for ff in ftype_points for jj in ff]
    Cs = [[cx[0] for cx in kwargs["C{}".format(ii)]] for ii in range(color_number)]
    g_parts = list(ftype_points) + \
              [[ii for ii in range(n) if ii not in ftype_union]]
    ppadd = 0 if is_graph else len(edges)
    g_verts = list(range(n+ppadd+color_number))
    g_parts.append(list(range(n, n+ppadd)))
    
    g_parts += [[n+ppadd+ii for ii in partition_j] for partition_j in order_partition]
    
    if is_graph:
        g_edges = list(edges)
        for ii in range(color_number):
            g_edges += [(xx, n+ii) for xx in Cs[ii]]
    else:
        g_edges = [(i+n,x) for i,b in enumerate(edges) for x in b]
        for ii in range(color_number):
            g_edges += [(xx, n+len(edges)+ii) for xx in Cs[ii]]
    g = Graph([g_verts, g_edges], format='vertices_and_edges')
    blocks = tuple(g.canonical_label(partition=g_parts).edges(labels=None, sort=True))
    return (n, tuple([len(xx) for xx in ftype_points]), blocks)

# This is also a default code for all color partition
def _generateCT(base_theory, k, order_partition, n):
    color_number = sum(len(xx) for xx in order_partition)
    BT = base_theory
    for xx in BT.generate_flags(n):
        unique = []
        edges = xx.blocks()['edges']
        
        for yy in itertools.product(range(color_number), repeat=int(n)):
            yy = list(yy)
            Cs = {"C{}".format(cc):[[ii] for ii, oo in enumerate(yy) if oo==cc] for cc in range(color_number)}
            iden = _identifyCT(k==2, order_partition, n, [], edges=edges, **Cs)
            if iden not in unique:
                unique.append(iden)
                Cs["edges"] = edges
                yield Cs

# to make the default codes work for this specific case:
# the generator:
# colors the elements of TGp (3-graphs without C5- and K4-), works on 3-uniform structures
# and the colors 0, 1, 2 are interchangeable (otherwise it would say [[0], [1], [2]]
def generate_colored(n):
    return _generateCT(TGp, 3, [[0, 1, 2]], n)

# same for the identifier. Colors are interchangeable.
def identify_colored(n, ftype_points, edges, C0, C1, C2):
    return _identifyCT(3, [[0, 1, 2]], n, ftype_points, edges=edges, C0=C0, C1=C1, C2=C2)

# CTGp is the colored variant of TGp (NoC5m)
CTGp = CombinatorialTheory("ColoredNoC5m", generate_colored, identify_colored, edges=3, C0=1, C1=1, C2=1)

# sanity check, the number of flags with size 4, 5, 6
# should be 18 132 2840
print(len(CTGp.generate(4)), len(CTGp.generate(5)), len(CTGp.generate(6)))

Process SpawnPoolWorker-44:
Process SpawnPoolWorker-40:
Process SpawnPoolWorker-41:
Process SpawnPoolWorker-43:
Traceback (most recent call last):
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
  File "/Users/maskat/local/sage-10.5.beta7/src/sage/algebras/flag_algebras.py", line 2

In [2]:
### This is where the actual calculation starts.
### This part just gives a standard upper bound on the number of edges

# degree equality
p2f4 = TGp.generate(4, TGp(2, ftype=[0, 1]))
degree_difference = p2f4[2]-p2f4[3]+p2f4[5]-p2f4[6]

pointed_edge = TGp(3, edges=[[0, 1, 2]], ftype=[0])

standard_assums = [degree_difference, -degree_difference, pointed_edge-1/4]

#run the optimizer
# TGp.optimize(TGp(3, edges=[[0, 1, 2]]), 7, positives=standard_assums, exact=True, denom=1024*1024, file="no_c5m_0")

In [3]:
### This code that minimizes k222

pointed_edge = TGp(3, edges=[[0, 1, 2]], ftype=[0])
edge = TGp(3, edges=[[0, 1, 2]])

# First k222 needs to be calculated, first the typed f222 is constructed
import itertools
be = [[0, 1, 2], [3, 4, 5], [0, 1, 5], [0, 2, 4], [1, 2, 3]]
ae = [[0, 4, 5], [1, 3, 5], [2, 3, 4]]
f222 = TGp(6, ftype=[0, 1, 2], edges=be)
for ii in [1, 2, 3]:
    for xx in itertools.combinations(ae, int(ii)):
        ad = TGp(6, ftype=[0, 1, 2], edges=be+list(xx))
        f222 = f222 + ad

# Then k222 is the projection of f222, this takes care of the automorphisms
k222 = f222.project()

degree_lower = pointed_edge - 1/4 + 1/100000
edge_upper = 25073/100000 - edge
uncolored_assums = [degree_lower, edge_upper]

# TGp.optimize(k222, 7, maximize=False, positives=uncolored_assums, exact=True, file="no_c5m_1", denom=1024*1024)

In [9]:
### This is the code that performs the calculations on the colored theory

# edge with (C)orrect colors
C = CTGp(3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1]], C2=[[2]])
# edge with (C)orrect colors (p)ointed
Cp = CTGp(3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1]], C2=[[2]], ftype=[0])

# edge with (B)ad colors (since color-blind, this includes the bad edges looking the wrong way too)
B = CTGp(3, edges=[[0, 1, 2]], C0=[[0], [1]], C1=[[2]])

# edge with (B)ad colors (p)ointed 
Bp = CTGp(3, edges=[[0, 1, 2]], C0=[[0], [2]], C1=[[1]], C2=[], ftype=[0])

# (M)issing edge with good colors
M = CTGp(3, edges=[], C0=[[0]], C1=[[1]], C2=[[2]])

# positivity assumptions
# each point, good edges are more than bad edges (divided by two due to the wrong color order)
# edge density is larger than 0.194 (from previous calculation)
TransversalDensity=190/1000
colored_assums = [Cp - Bp/2, C - TransversalDensity]

# optimal construction and its derivatives
optim = CTGp.blowup_construction(6, 3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1]], C2=[[2]], symmetric=True, symbolic=True)
ssoptim = optim.set_sum()
der_optims = ssoptim.derivatives([1/3, 1/3])

# bad is less than missing, proven by (B)ad (M)inum (M)issing is at most 0.
CTGp.optimize(B + (-99/100)*M, 6, maximize=True, positives=colored_assums, 
              exact=True, construction=der_optims, denom=2048, file="no_c5m_2")

100%|██████████████████████████████████████████| 28/28 [00:00<00:00, 586.88it/s]


Base flags generated, their number is 2840
The relevant ftypes are constructed, their number is 20
Block sizes before symmetric/asymmetric change is applied: [35, 57, 82, 123, 123, 123, 40, 60, 60, 60, 60, 60, 20, 30, 30, 30, 30, 30, 30, 30]


Done with mult table for Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3]], C0=[[0]], C1=[[1]], C2=[[2], [3]]: : 20it [00:00, 2581.35it/s]


Tables finished


Done with positivity constraint 1: 100%|██████████| 2/2 [00:01<00:00,  1.46it/s]


Constraints finished
Adjusting table with kernels from construction
Running SDP after kernel correction. Used block sizes are [23, 9, 29, 24, 12, 68, 33, 87, 29, 91, 43, 80, 14, 26, 21, 39, 39, 21, 39, 21, 39, 21, 33, 27, 12, 8, 21, 9, 18, 12, 13, 17, 24, 6, 17, 13, 30, 12, 15, -2840, -50]
Process stopped by signal.
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Failure: return code is 10 
Primal objective value: 0.0000000e+00 
Dual objective value: 0.0000000e+00 
Relative primal infeasibility: 3.55e+02 
Relative dual infeasibility: 1.18e+03 
Real Relative Gap: 0.00e+00 
XZ Relative Gap: 8.73e+06 
DIMACS error measures: 2.10e+03 0.00e+00 2.43e+03 0.00e+00 0.00e+00 8.73e+06
Starting the rounding of the result
Rounding X matrices
Calculating resulting bound


100%|███████████████████████████████████████████| 20/20 [02:18<00:00,  6.95s/it]


Final rounded bound is 6516377/76800


{'result': 6516377/76800,
 'X matrices': [(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 58629/2048, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 58629/2048, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10545/256, 16449/1024, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10545/256, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 58629/2048, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 58629/2048, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10545/256, 16449/1024, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
## Verify the certificates, 
## can be run without running the long calculations above
## still takes a rather long time, as the matrices are provided
## in the X = P L D L.T P.T format, and it takes a while to perform
## the multiplications


# For the 0th part
p2f4 = TGp.generate(4, TGp(2, ftype=[0, 1]))
degree_difference = p2f4[2]-p2f4[3]+p2f4[5]-p2f4[6]
pointed_edge = TGp(3, edges=[[0, 1, 2]], ftype=[0])
standard_assums = [degree_difference, -degree_difference, pointed_edge-1/4]
TGp.verify("no_c5m_0", TGp(3, edges=[[0, 1, 2]]), 7, positives=standard_assums)

# For the 1st part
pointed_edge = TGp(3, edges=[[0, 1, 2]], ftype=[0])
edge = TGp(3, edges=[[0, 1, 2]])
import itertools
be = [[0, 1, 2], [3, 4, 5], [0, 1, 5], [0, 2, 4], [1, 2, 3]]
ae = [[0, 4, 5], [1, 3, 5], [2, 3, 4]]
f222 = TGp(6, ftype=[0, 1, 2], edges=be)
for ii in [1, 2, 3]:
    for xx in itertools.combinations(ae, int(ii)):
        ad = TGp(6, ftype=[0, 1, 2], edges=be+list(xx))
        f222 = f222 + ad
k222 = f222.project()
degree_lower = pointed_edge - 1/4 + 1/100000
edge_upper = 25073/100000 - edge
uncolored_assums = [degree_lower, edge_upper]
TGp.verify("no_c5m_1", k222, 7, maximize=False, positives=uncolored_assums)

# For the 2nd part
C = CTGp(3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1]], C2=[[2]])
Cp = CTGp(3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1]], C2=[[2]], ftype=[0])
B = CTGp(3, edges=[[0, 1, 2]], C0=[[0], [1]], C1=[[2]])
Bp = CTGp(3, edges=[[0, 1, 2]], C0=[[0], [2]], C1=[[1]], C2=[], ftype=[0])
M = CTGp(3, edges=[], C0=[[0]], C1=[[1]], C2=[[2]])
colored_assums = [Cp - Bp/2, C - TransversalDensity]
optim = CTGp.blowup_construction(6, 3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1]], C2=[[2]], symmetric=True, symbolic=True)
ssoptim = optim.set_sum()
der_optims = ssoptim.derivatives([1/3, 1/3])
CTGp.verify("no_c5m_2", B + (-99/100)*M, 6, maximize=True, positives=colored_assums, construction=der_optims)